<a href="https://colab.research.google.com/github/dibend/Colab/blob/main/FRED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly gradio scikit-learn

In [ ]:
import pandas as pd
import plotly.express as px
import requests
import gradio as gr
import io
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import timedelta

def get_fred_data(series_id):
    url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(data), parse_dates=['DATE'])
    df.columns = ['Date', 'Value']
    return df

def plot_fred_data(df, title):
    fig = px.line(df, x='Date', y='Value', title=title, labels={'Value': 'Actual Values'})
    if 'Future_Predictions' in df.columns:
        fig.add_scatter(x=df['Date'], y=df['Future_Predictions'], mode='lines', name='Future Predictions', line=dict(color='red'))
    fig.update_xaxes(rangeslider_visible=True)
    return fig

def linear_regression_future(df, future_days):
    df['Days_Since_Start'] = (df['Date'] - df['Date'].min()).dt.days
    X = df[['Days_Since_Start']]
    y = df['Value']
    model = LinearRegression()
    model.fit(X, y)
    df['LR_Predicted'] = model.predict(X)

    last_date = df['Date'].max()
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=future_days)
    future_days_since_start = (future_dates - df['Date'].min()).days
    future_df = pd.DataFrame({'Date': future_dates, 'Days_Since_Start': future_days_since_start})
    future_df['Value'] = model.predict(future_df[['Days_Since_Start']])

    combined_df = pd.concat([df, future_df], ignore_index=True)
    combined_df['Future_Predictions'] = np.nan
    combined_df.loc[len(df):, 'Future_Predictions'] = combined_df.loc[len(df):, 'Value']
    return combined_df

def lstm_future(df, layers, units, epochs, batch_size, future_days, dropout_rate):
    df['Value'] = df['Value'].astype(float)
    data = df[['Value']].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    sequence_length = 10
    X = []
    y = []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i + sequence_length])
        y.append(scaled_data[i + sequence_length])
    X, y = np.array(X), np.array(y)

    model = Sequential()
    for _ in range(layers - 1):
        model.add(LSTM(units=units, return_sequences=True, input_shape=(sequence_length, 1)))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2)

    predictions = model.predict(X)
    predictions = scaler.inverse_transform(predictions)
    df['LSTM_Predicted'] = np.nan
    df.iloc[sequence_length:, df.columns.get_loc('LSTM_Predicted')] = predictions.flatten()

    last_sequence = scaled_data[-sequence_length:]
    future_preds = []
    for _ in range(future_days):
        next_pred = model.predict(last_sequence.reshape(1, sequence_length, 1))
        future_preds.append(next_pred[0, 0])
        last_sequence = np.append(last_sequence[1:], next_pred)

    future_preds = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))
    last_date = df['Date'].max()
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=future_days)
    future_df = pd.DataFrame({'Date': future_dates, 'Value': future_preds.flatten()})
    combined_df = pd.concat([df, future_df], ignore_index=True)
    combined_df['Future_Predictions'] = np.nan
    combined_df.loc[len(df):, 'Future_Predictions'] = combined_df.loc[len(df):, 'Value']

    return combined_df

def display_data(series_id):
    df = get_fred_data(series_id)
    stats = df.describe().reset_index()
    plot = plot_fred_data(df, f'FRED Data for {series_id}')
    return stats, plot

def run_linear_regression(series_id, future_days):
    df = get_fred_data(series_id)
    df_lr = linear_regression_future(df, future_days)
    lr_plot = plot_fred_data(df_lr, f'{series_id} - Linear Regression with Future Predictions')
    return lr_plot

def run_lstm(series_id, layers, units, epochs, batch_size, future_days, dropout_rate):
    df = get_fred_data(series_id)
    df_lstm = lstm_future(df, layers, units, epochs, batch_size, future_days, dropout_rate)
    lstm_plot = plot_fred_data(df_lstm, f'{series_id} - LSTM with Future Predictions')
    return lstm_plot

series_id_choices = [
    ('Gross Domestic Product', 'GDP'),
    ('Consumer Price Index for All Urban Consumers', 'CPIAUCSL'),
    ('Unemployment Rate', 'UNRATE'),
    ('Federal Funds Rate', 'FEDFUNDS'),
    ('10-Year Treasury Constant Maturity Rate', 'DGS10'),
    ('M2 Money Stock', 'M2'),
    ('Total Nonfarm Payrolls', 'PAYEMS'),
    ('Personal Consumption Expenditures', 'PCE'),
    ('Industrial Production Index', 'INDPRO'),
    ('Housing Starts', 'HOUST'),
    ('Retail Sales', 'RSAFS'),
]

with gr.Blocks() as demo:
    with gr.Tabs():
        with gr.TabItem("Statistics and Series"):
            series_id_input1 = gr.Dropdown(choices=series_id_choices, value='GDP', label="FRED Series ID")
            stats_button = gr.Button("Generate Statistics and Plot")
            stats_output = gr.DataFrame(label="Series Statistics")
            series_plot_output = gr.Plot(label="Series Plot")
            stats_button.click(display_data, inputs=series_id_input1, outputs=[stats_output, series_plot_output])

        with gr.TabItem("Linear Regression"):
            series_id_input2 = gr.Dropdown(choices=series_id_choices, value='GDP', label="FRED Series ID")
            future_days_lr_input = gr.Slider(minimum=1, maximum=365, step=1, value=30, label="Days to Predict (Linear Regression)")
            lr_button = gr.Button("Run Linear Regression")
            lr_plot_output = gr.Plot(label="Linear Regression Plot")
            lr_button.click(run_linear_regression, inputs=[series_id_input2, future_days_lr_input], outputs=lr_plot_output)

        with gr.TabItem("LSTM"):
            series_id_input3 = gr.Dropdown(choices=series_id_choices, value='GDP', label="FRED Series ID")
            layers_input = gr.Slider(minimum=1, maximum=5, step=1, value=2, label="LSTM Layers")
            units_input = gr.Slider(minimum=10, maximum=100, step=10, value=50, label="LSTM Units")
            epochs_input = gr.Slider(minimum=10, maximum=100, step=10, value=20, label="LSTM Epochs")
            batch_size_input = gr.Slider(minimum=10, maximum=100, step=10, value=32, label="LSTM Batch Size")
            dropout_rate_input = gr.Slider(minimum=0.0, maximum=0.5, step=0.1, value=0.2, label="Dropout Rate")
            future_days_lstm_input = gr.Slider(minimum=1, maximum=365, step=1, value=30, label="Days to Predict (LSTM)")
            lstm_button = gr.Button("Run LSTM")
            lstm_plot_output = gr.Plot(label="LSTM Plot")
            lstm_button.click(run_lstm, inputs=[series_id_input3, layers_input, units_input, epochs_input, batch_size_input, future_days_lstm_input, dropout_rate_input], outputs=lstm_plot_output)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6c3becc399159a1ad1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Epoch 1/20
10/10 - 4s - loss: 0.0799 - 4s/epoch - 365ms/step
Epoch 2/20
10/10 - 0s - loss: 0.0156 - 89ms/epoch - 9ms/step
Epoch 3/20
10/10 - 0s - loss: 0.0059 - 91ms/epoch - 9ms/step
Epoch 4/20
10/10 - 0s - loss: 0.0033 - 92ms/epoch - 9ms/step
Epoch 5/20
10/10 - 0s - loss: 0.0032 - 95ms/epoch - 9ms/step
Epoch 6/20
10/10 - 0s - loss: 0.0020 - 95ms/epoch - 9ms/step
Epoch 7/20
10/10 - 0s - loss: 0.0017 - 92ms/epoch - 9ms/step
Epoch 8/20
10/10 - 0s - loss: 0.0016 - 92ms/epoch - 9ms/step
Epoch 9/20
10/10 - 0s - loss: 0.0018 - 93ms/epoch - 9ms/step
Epoch 10/20
10/10 - 0s - loss: 0.0020 - 91ms/epoch - 9ms/step
Epoch 11/20
10/10 - 0s - loss: 0.0011 - 92ms/epoch - 9ms/step
Epoch 12/20
10/10 - 0s - loss: 0.0014 - 85ms/epoch - 9ms/step
Epoch 13/20
10/10 - 0s - loss: 0.0012 - 84ms/epoch - 8ms/step
Epoch 14/20
10/10 - 0s - loss: 0.0015 - 87ms/epoch - 9ms/step
Epoch 15/20
10/10 - 0s - loss: 0.0017 - 89ms/epoch - 9ms/step
Epoch 16/20
10/10 - 0s - loss: 0.0018 - 90ms/epoch - 9ms/step
Epoch 17/20
10/10

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Epoch 1/20
25/25 - 3s - loss: 0.0410 - 3s/epoch - 135ms/step
Epoch 2/20
25/25 - 0s - loss: 0.0115 - 258ms/epoch - 10ms/step
Epoch 3/20
25/25 - 0s - loss: 0.0103 - 248ms/epoch - 10ms/step
Epoch 4/20
25/25 - 0s - loss: 0.0092 - 243ms/epoch - 10ms/step
Epoch 5/20
25/25 - 0s - loss: 0.0095 - 244ms/epoch - 10ms/step
Epoch 6/20
25/25 - 0s - loss: 0.0090 - 248ms/epoch - 10ms/step
Epoch 7/20
25/25 - 0s - loss: 0.0083 - 240ms/epoch - 10ms/step
Epoch 8/20
25/25 - 0s - loss: 0.0086 - 241ms/epoch - 10ms/step
Epoch 9/20
25/25 - 0s - loss: 0.0079 - 246ms/epoch - 10ms/step
Epoch 10/20
25/25 - 0s - loss: 0.0082 - 235ms/epoch - 9ms/step
Epoch 11/20
25/25 - 0s - loss: 0.0081 - 244ms/epoch - 10ms/step
Epoch 12/20
25/25 - 0s - loss: 0.0074 - 236ms/epoch - 9ms/step
Epoch 13/20
25/25 - 0s - loss: 0.0070 - 228ms/epoch - 9ms/step
Epoch 14/20
25/25 - 0s - loss: 0.0068 - 212ms/epoch - 8ms/step
Epoch 15/20
25/25 - 0s - loss: 0.0074 - 219ms/epoch - 9ms/step
Epoch 16/20
25/25 - 0s - loss: 0.0064 - 218ms/epoch - 9m

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

